<a href="https://colab.research.google.com/github/Khawla-1/ResponseAssessment/blob/master/AssessmentResponse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# In this document I'll give a response for the Technical Assessment

**Task 1**: Implement Partial Cross Entropy Loss
Partial Cross Entropy (PCE)

In [ ]:
import torch
import torch.nn as nn

class PartialCrossEntropyLoss(nn.Module):

    def __init__(self):
        super(PartialCrossEntropyLoss, self).__init__()

    def forward(self, predictions, labels, mask):

        # Flatten tensors for mask application
        predictions = predictions.permute(0, 2, 3, 1).reshape(-1, predictions.size(1))  # [N, C]
        labels = labels.view(-1)  # [N]
        mask = mask.view(-1)  # [N]

        # Mask predictions and labels
        predictions = predictions[mask == 1]
        labels = labels[mask == 1]

        # Standard Cross Entropy Loss
        loss = nn.CrossEntropyLoss()(predictions, labels)
        return loss


### **Task 2**: Task 2: Find Remote Sensing Data, Simulate Point Labels, and Integrate Loss

* Objective: Test the PartialCrossEntropyLoss with remote sensing data and simulate sparse annotations.

Implementation:

1 - Data Loading: start by remote sensing dataset like EuroSAT or DeepGlobe. then simulate point labels with a random sampling strategy.

2- Integrate Partial Cross Entropy Loss: Implement it into a semantic segmentation framework (e.g., U-Net).


In [ ]:
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader
import torchvision.transforms as T
import numpy as np

# Load remote sensing dataset
def load_remote_sensing_data():
    transform = T.Compose([T.Resize((256, 256)), T.ToTensor()])
    dataset = VOCSegmentation(root='./data', year='2012', image_set='train', download=True, transform=transform)
    return DataLoader(dataset, batch_size=4, shuffle=True)

# Simulate sparse point labels
def simulate_point_labels(segmentation_labels, num_points=50):

    batch_size, height, width = segmentation_labels.shape
    sparse_labels = torch.zeros_like(segmentation_labels)
    mask = torch.zeros_like(segmentation_labels)

    for batch_idx in range(batch_size):
        for _ in range(num_points):
            x, y = np.random.randint(0, height), np.random.randint(0, width)
            sparse_labels[batch_idx, x, y] = segmentation_labels[batch_idx, x, y]
            mask[batch_idx, x, y] = 1
    return sparse_labels, mask




## **Task 3** Design Experiments


**Purpose**:

* Assess the impact of labeled point density (10, 50, 100 points per image).
* Test semi-supervised learning techniques like self-training or consistency regularization.

**Hypothesis**:
* More labeled points improve segmentation performance.
* Semi-supervised approaches enhance performance by leveraging unlabeled regions.

**Experimental Process**:

* Train U-Net using PartialCrossEntropyLoss with different point densities.
* Implement pseudo-labeling for unlabeled regions.

In [ ]:
from tqdm import tqdm
import torch.optim as optim

# Define U-Net or any segmentation model
class UNet(nn.Module):
    # Define model architecture
    def __init__(self):
        super(UNet, self).__init__()
        # ...

    def forward(self, x):
        # ...
        return x

# Training loop
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, point_density=50):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for images, labels in tqdm(dataloader):
            # Simulate sparse annotations
            sparse_labels, mask = simulate_point_labels(labels, num_points=point_density)

            # Forward pass
            predictions = model(images)

            # Loss calculation
            loss = criterion(predictions, sparse_labels, mask)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}")

# Experiment
def experiment():
    dataloader = load_remote_sensing_data()
    model = UNet()
    criterion = PartialCrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Vary point density and observe performance
    for point_density in [10, 50, 100]:
        print(f"\nTraining with {point_density} labeled points per image")
        train_model(model, dataloader, criterion, optimizer, num_epochs=5, point_density=point_density)


In [ ]:
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader
import torchvision.transforms as T

# Load remote sensing dataset with proper transformations
def load_remote_sensing_data():
    transform = T.Compose([
        T.Resize((256, 256)),  # Resize images and labels
        T.ToTensor(),          # Convert images to tensors
    ])

    target_transform = T.Compose([
        T.Resize((256, 256)),  # Resize segmentation masks
        T.PILToTensor(),       # Convert masks to tensors
    ])

    dataset = VOCSegmentation(
        root='./data',
        year='2012',
        image_set='train',
        download=True,
        transform=transform,
        target_transform=target_transform
    )
    return DataLoader(dataset, batch_size=4, shuffle=True)

# Example usage
dataloader = load_remote_sensing_data()
images, labels = next(iter(dataloader))
print(images.shape, labels.shape)  # Validate tensor shapes


Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
torch.Size([4, 3, 256, 256]) torch.Size([4, 1, 256, 256])
